In [171]:
import pandas as pd
import re
import numpy as np
pd.set_option("display.max_columns",101)

In [135]:
data = pd.read_csv('test_ds.csv')

In [183]:
data.head()

,additional_info,rb,name,id,summary_title,location,current_job_company,current_job_title,current_job_desc,current_job_duration,prev_job_company,prev_job_title,prev_job_desc,prev_job_duration,total_exp_durations,education1_title,education1_college,education1_duration,education2_title,education2_college,education2_duration,education3_title,education3_college,education3_duration,skils,links,certifications,awards,work_auth,current_job_duration_months,prev_job_duration_months,total_exp_durations_months,work_authp,prev_job_desc_clean
0,"['TECHNICAL SKILLS\xa0', <br/>, '\xa0', <br/>,...",NaN,Tracy-Ruan,c47f7ac095973653?sp=0,Data Scientist Intern,"San Francisco, CA",NaN,Data Scientist Intern,['• Prepared customer behavior datasets for cl...,August 2017 to November 2017,NaN,Web Designer Assistant,['• Maintained university website (HTML ...,September 2015 to January 2016,"['August 2017 to November 2017', 'September 20...",B.S. in Data Science,University of San Francisco,January 2013 to May 2017,NaN,NaN,NaN,NaN,NaN,NaN,"[['JAVASCRIPT (Less than 1 year)'], ['PYTHON (...",['https://github.com/tracyruan007'],[],[],Authorized to work in the US for any employer,3,4,8,1.0,"[maintained, university, website, html, cs, in..."
1,"['TECHNICAL SKILLS:\xa0', <br/>, 'Languages ...",NaN,Sai-Nadimpalli,19e0d35744cc56a6?sp=0,Junior Data Scientist,"Kansas City, MO",['Sprint Corporation'],Junior Data Scientist,['• Developed a Hybrid Recommendation System f...,May 2017 to August 2017,['IBM'],Software Development Engineer Intern,['• Developed Web and Android Applications for...,September 2015 to January 2016,"['May 2017 to August 2017', 'September 2015 to...",Master of Science in Computer Science in Compu...,University of Missouri,May 2018,Technology,K L University,May 2016,NaN,NaN,NaN,"[['C (Less than 1 year)'], ['CSS (Less than 1 ...",[],[],[],NaN,3,4,7,NaN,"[developed, web, android, application, ibm, bl..."
2,"['TECHNICAL SKILLS\xa0', <br/>, '• Proficien...",NaN,Nick-Shi,accfd33784428f69?sp=0,Data Scientist Intern,"Malibu, CA",['MarketPsych Data'],Data Scientist Intern,['• Manipulated and interpreted insights fro...,February 2018 to Present,NaN,Technical Consultant,['• Manipulated 12 year historical option da...,January 2018 to Present,"['February 2018 to Present', 'January 2018 to ...",Master of Science in Applied Finance,"Pepperdine University, Graziadio School of Bus...",December 2017,Master of Science in Computing,Imperial College London,October 2016,Bachelor of Science in Software Engineering,"The University of Nottingham, School of Comput...",July 2015,"[['PYTHON (Less than 1 year)'], ['SQL (Less th...",['http://www.linkedin.com/in/nick-shi'],[],[],Sponsorship required to work in the US,1,2,12,0.0,"[manipulated, 12, year, historical, option, da..."
3,"['TECHNICAL SKILLS\xa0', <br/>, 'Relevant Cour...",NaN,Harsh-Mehta,2a4af24d87cca9cd?sp=0,Data Scientist,"Bloomington, IN",['Indiana University'],Data Scientist,['• Implemented probabilistic character recogn...,November 2017 to November 2017,['University of Mumbai'],Data Analyst,['• Performed analysis of supermarket data by ...,June 2017 to June 2017,"['November 2017 to November 2017', 'June 2017 ...",Master's in Data Science,"Indiana University, School of Informatics",May 2019,Bachelor's in Information Technology,University of Mumbai,June 2017,NaN,NaN,NaN,"[['APACHE HBASE (Less than 1 year)'], ['ARTIFI...","['http://github.com/humehta/Projects.git', 'ht...",[],"['TSEC Hall of Fame', 'TSEC Deparment Ambassad...",Authorized to work in the US for any employer,0,0,1,1.0,"[performed, analysis, supermarket, data, loadi..."
4,"['SKILLS\xa0', <br/>, '\xa0', <br/>, 'SOFTWARE...",NaN,Daniel-Wu,246fa163d0b35d5b?sp=0,DATA SCIENTIST,"Brooklyn, NY",['MAPSCorps'],DATA SCIENTIST,['Acted as a Chinese translator when collectin...,July 2016 to August 2016,['NEUROLOGY CLINIC FRONT DESK'],Front Desk,['• Translations from English to Chinese (Cant...,May 2016 to July 2016,"['July 2016 to August 2016', 'May 2016 to July...",E

### Fill NA values

In [137]:
data['current_job_duration']=data.current_job_duration.fillna(0)
data['prev_job_duration']=data.prev_job_duration.fillna(0)
data['total_exp_durations']=data.total_exp_durations.fillna(0)

### Convert dates in duration

In [139]:
months = {'January' : 1,'February' : 2, 'March':3, "April":4, 'May':5, 'June':6, 'July':7, 'August':8, 'September':9, 'October':10, 'November':11, 'December':12}
def convert_dates(date_string):
    if date_string == 0:
        return 0
    try:
        from_, to_ = date_string.split(' to ')
        if len(from_.split(' '))==1:
            from_ = 'July '+from_
        from_month, from_year = from_.split(' ')
        if to_.strip() == 'Present':
            to_ = 'March 2018'

        if len(to_.split(' '))==1:
            to_ = 'July '+to_
        to_month, to_year = to_.split(' ')
        return (int(to_year)-int(from_year))*12 + months[to_month]- months[from_month]
    except:
        return 0

def get_total_exp(exp_str):
    exp_list=[]
    if exp_str ==0:
        return 0
    try:
        exp_list = exp_str.replace('[','').replace(']','').replace("'",'').split(',')
    except:
        return 0
    if exp_list is None:
        return 0
    exp=0
    try:
        for x in exp_list:
            exp = exp + convert_dates(x.strip())
        return exp
    except:
        return exp
    

In [140]:
data['current_job_duration_months']= data.current_job_duration.apply(convert_dates)
data['prev_job_duration_months']= data.prev_job_duration.apply(convert_dates)
data['total_exp_durations_months'] = data.total_exp_durations.apply(get_total_exp)

In [144]:
data.work_auth.unique()

array(['Authorized to work in the US for any employer', nan,
       'Sponsorship required to work in the US'], dtype=object)

#### Work Authorization

In [154]:
def process_work_auth(x):
    if 'Authorized' in str(x):
        return 1
    elif 'Sponsorship' in str(x):
        return 0
    else:
        return None

In [155]:
data['work_authp'] = data.work_auth.apply(process_work_auth)

### LDA for previous work title

In [166]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer

In [167]:
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()

In [250]:
def clean(doc):
    doc = re.sub('<br/>', '',str(doc))
    doc = re.sub('\\xa0', '', doc)
    doc= re.sub('[^A-Za-z0-9\ ]+', '', doc)
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized.split()

In [251]:
doc_clean= data.prev_job_desc.apply(clean)

In [252]:
import gensim
from gensim import corpora

In [254]:
dictionary = corpora.Dictionary(doc_clean)

In [255]:
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [256]:
Lda= gensim.models.ldamodel.LdaModel

In [257]:
def get_coherence(model):
    coh=0
    for x in model:
        coh+=x[1]
    return coh/len(model)
#get_coherence(ldamodel.top_topics(texts=doc_clean))

In [258]:
ldamodel = Lda(doc_term_matrix, num_topics=9, id2word = dictionary, passes=100)
this_coh = get_coherence(ldamodel.top_topics(texts=doc_clean))

In [406]:
data.to_csv('data_clean_1.csv',index=False)

In [298]:
def clean_titles(doc):
    doc = re.sub('[^A-Za-z0-9\ ]+', '',str(doc).lower())
    return doc

In [299]:
title_list = []
for i in data.prev_job_title.apply(clean_titles):
    title_list = title_list + i.split()
    
df_keys= pd.DataFrame({'keyword':title_list})

In [374]:
df_keys.keyword.value_counts()[60:120]

quantitative     82
statistician     80
founder          79
vice             79
information      78
clinical         78
in               78
quality          77
cofounder        73
contractor       73
database         72
head             71
staff            70
ii               69
summer           69
member           68
risk             64
analysis         64
department       64
financial        64
web              62
hadoop           61
leader           60
system           59
solutions        59
coordinator      59
internship       59
officer          59
tutor            58
customer         58
administrator    58
independent      57
application      57
consulting       56
strategy         56
services         54
market           53
teacher          51
bi               51
lecturer         50
modeling         50
account          49
executive        48
process          47
technician       47
design           44
digital          44
lab              43
enterprise       43
the              43


In [401]:
def get_job_cat(doc):
    doc = clean_titles(doc)
    if doc is None:
        return 'Others'
    #Student:
    if 'lab' in doc or 'internship' in doc or'intern' in doc or 'student' in doc or 'teaching' in doc or 'graduate' in doc or 'summer' in doc or 'research' in doc:
        return 'Student'
    if 'machine' in doc or 'learning' in doc:
        return 'Data Scientist'
    if 'data' in doc and 'scientist' in doc:
        return 'Data Scientist'
    if 'data' in doc and 'researcher' in doc:
        return 'Data Scientist'
    if 'data' in doc and 'research' in doc:
        return 'Data Scientist'
    if 'science' in doc:
        return 'Data Scientist'
    if 'database' in doc or 'sql' in doc:
        return 'Data Analyst'
    if 'analyst' in doc or 'analytics' in 'doc' or 'analysis' in doc:
        if 'data' in doc:
            return 'Data Analyst'
        else:
            return 'Business Analyst'
    if 'postdoctoral' in doc or 'fellow' in doc or 'ph.d' in doc or 'phd' in doc:
        return 'Data Scientist'
    if 'data' in doc:
        return 'Data Analyst'
    if 'marketing' in doc or 'risk' in doc or 'financial' in doc or 'market' in doc:
        return 'Business Analyst'
    if 'business' in doc:
        return 'Business Analyst'
    if 'systems' in doc or 'system' in doc:
        return 'Software Engg' 
    if 'big' in doc or 'hadoop' in doc:
        return 'Data Scientist'
    if 'big' in doc or 'statistical' in doc or 'statistics' in doc or 'statistician' in doc:
        return 'Data Scientist'
    if 'software' in doc or 'developer' in doc or 'development' in doc or 'architect' in doc or 'development' in doc:
        return 'Software Engg'
    if 'engineering' in doc or 'programmer' in doc or 'technology' in doc or 'web' in doc:
        return 'Software Engg'
    if 'consulting' in doc or 'consultant' in doc or 'manager' in doc or 'project' in doc or 'product' in doc or 'management' in doc or 'director' in doc:
        return 'Project Manager'
    else:
        return 'Others'

In [397]:
get_job_cat('Software Development Engineer Intern')

'Student'

In [402]:
data['prev_job_title_cats'] = data.prev_job_title.apply(get_job_cat)

In [260]:
ldamodel.print_topics(num_topics=15)

[(0,
  '0.020*"data" + 0.017*"system" + 0.011*"application" + 0.011*"database" + 0.011*"using" + 0.010*"xa0" + 0.010*"sql" + 0.009*"server" + 0.009*"design" + 0.008*"project"'),
 (1,
  '0.018*"research" + 0.013*"nan" + 0.013*"analysis" + 0.013*"student" + 0.009*"xa0" + 0.009*"developed" + 0.008*"course" + 0.007*"method" + 0.006*"simulation" + 0.006*"statistical"'),
 (2,
  '0.055*"data" + 0.023*"hadoop" + 0.021*"using" + 0.019*"hive" + 0.016*"spark" + 0.010*"hdfs" + 0.009*"aws" + 0.009*"worked" + 0.009*"pig" + 0.009*"big"'),
 (3,
  '0.015*"risk" + 0.011*"project" + 0.010*"health" + 0.010*"management" + 0.010*"clinical" + 0.007*"system" + 0.006*"study" + 0.006*"patient" + 0.005*"medical" + 0.005*"credit"'),
 (4,
  '0.010*"trading" + 0.009*"sqoop" + 0.009*"stock" + 0.007*"cassandra" + 0.006*"time" + 0.006*"trade" + 0.005*"built" + 0.005*"price" + 0.004*"million" + 0.004*"strategy"'),
 (5,
  '0.063*"data" + 0.027*"using" + 0.027*"model" + 0.020*"analysis" + 0.012*"r" + 0.010*"python" + 0.0

In [261]:
prev_job_type_dict= {0:'Sql Analyst', 1: 'Research Student', 2: 'Big Data', 3: 'Other', 4:'Other', 5:'Machine Learning', 6:'Softwre engineer', 7:'Machine Learning',8:'Product Manager'}

In [267]:
ldamodel.get_document_topics(dictionary.doc2bow(data.prev_job_desc_clean.iloc[2]))

[(0, 0.28887993), (4, 0.2697291), (5, 0.3683205), (8, 0.060438965)]